In [1]:
import boto3
from scraper.webscraper import Scraper

scraper = Scraper() #call scraper class



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been 

Webpage loaded successfully


In [ ]:
from scraper.webscraper import Scraper

scraper = Scraper() #call scraper class
scraper.accept_cookies()
scraper.search_bar('Kingdom Hearts') #add search keyword here
container = scraper.find_container()
scraper.collect_page_links()
src = scraper.find_images(container= container)
scraper.upload_images(src)

In [ ]:
#add files to bucket 
import boto3
import os
import tempfile
import sqlalchemy #create an engine which connects local machine/python script with database
from sqlalchemy import create_engine 

client = boto3.client('s3')
bucket = os.environ.get('DB_BUCKET')
sourcepath = "images"
source = os.listdir(sourcepath)

for file in source:
    '''
     specify location of files in images directory, otherwise it will try to look 
     for images in current working directory
    '''

    with open(f'images/{file}', "rb") as f: #ensures each file is looked at individually 
       client.upload_fileobj(f, bucket, file) #image files will be added to bucket
        
    #print(file)
#source[0]
 

In [2]:
import os
import sqlalchemy #create an engine which connects local machine/python script with database
from sqlalchemy import create_engine 

DATABASE_TYPE = os.environ.get('DB_DATABASE_TYPE')
DBAPI = os.environ.get('DB_DBAPI') #database API - API to connect Python with database
#use AWS details to connect database - saved in Environment Variables
HOST = os.environ.get('DB_HOST') #endpoint
USER = os.environ.get('DB_USER') #username
PASSWORD = os.environ.get('DB_PASS')
DATABASE = os.environ.get('DB_DATABASE')
PORT = os.environ.get('DB_PORT')

engine = create_engine(f'{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

In [3]:
engine.connect()

In [ ]:
#upload json files to bucket
import boto3

client = boto3.client('s3')
bucket = os.environ.get('DB_BUCKET')
'''specify directory where data.json is located
    upload_fileobj is used where the file has already been opened
    otherwise use upload_file'''
#with open("raw_data/data.json", "rb") as file: #also works
 #  response = client.upload_fileobj(file, bucket, "data.json") 
client.upload_file("raw_data/data.json", bucket, "data.json")
client.upload_file("raw_data/image_data.json", bucket, "image_data.json")

In [ ]:
#create folders and upload files into respective folders in bucket
import boto3

client = boto3.client('s3')
#create folder in s3 bucket
bucket = os.environ.get('DB_BUCKET')

#folders
image_folder = "images"
data_folder = "raw_data"

client.put_object(Bucket=bucket, Key=(image_folder+'/'))
client.put_object(Bucket=bucket, Key=(data_folder+'/'))

#upload files into s3 bucket folders
client.upload_file("raw_data/data.json", bucket, f"{data_folder}/data.json") #saves data.json file in folder
client.upload_file("raw_data/image_data.json", bucket, f"{data_folder}/image_data.json")

sourcepath = "images"
source = os.listdir(sourcepath)

#upload images to s3 bucket folder
for file in source:
    with open(f'images/{file}', "rb") as f: 
       '''specify location of files in images directory, otherwise it will try to look 
            for images in current working directory'''
       client.upload_fileobj(f, bucket, f'{image_folder}/'+file)

In [ ]:
#save data.json as pandas dataframe and sql table
import json
import pandas as pd
from sqlalchemy import create_engine
with open('raw_data/data.json') as json_file:
    data = json.load(json_file) #load json file
    print("Type:", type(data))
    df_products = pd.DataFrame(data) #save data in json file in dataframe
sql_table = df_products.to_sql("SE_products", con=scraper.engine, if_exists='replace', index=False) #upload pandas dataframe to sql table 
                                                                                                   #use replace instead of append if changing everything in sql table
sql_table #prints number of entries

In [ ]:
#save image_data.json as pandas dataframe and sql table
import json
import pandas as pd
from sqlalchemy import create_engine
with open('raw_data/image_data.json') as json_file:
    data = json.load(json_file) #load json file
    print("Type:", type(data))
    df_images = pd.DataFrame(data) #save data in json file in dataframe
sql_table_i = df_images.to_sql("SE_product_images", con=scraper.engine, if_exists='replace', index=False) #upload pandas dataframe to sql table 
                                                                                                   #use replace instead of append if changing everything in sql table

In [ ]:
df_products

In [ ]:
#merge tables on product_uuid column
import pandas as pd

se_products = pd.merge(df_products, df_images, on= 'product_uuid', how="outer")
se_table = se_products.to_sql("SE_products_final", con=scraper.engine, if_exists='replace', index=False) #upload pandas dataframe to sql table 

se_products